In [6]:
pip install cupy-cuda12x opencv-python

  Using cached cupy_cuda12x-13.6.0-cp312-cp312-manylinux2014_x86_64.whl.metadata (2.4 kB)
  Using cached fastrlock-0.8.3-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_28_x86_64.whl.metadata (7.7 kB)
Using cached cupy_cuda12x-13.6.0-cp312-cp312-manylinux2014_x86_64.whl (112.9 MB)
Using cached fastrlock-0.8.3-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_28_x86_64.whl (53 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [cupy-cuda12x] [cupy-cuda12x]
Note: you may need to restart the kernel to use updated packages.


In [8]:
import cv2
import os

img_path = "/workspace/scan_images/X51005230617.jpg"

if not os.path.exists(img_path):
    print(f"File not found: {img_path}")
else:
    img = cv2.imread(img_path)
    if img is None:
        print(f"Failed to read image: {img_path}")
    else:
        resized = cv2.resize(img, (256, 256))
        print("Resize successful")


Resize successful


In [3]:
import torch
import cv2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type != 'cuda':
    print("Không có GPU, không thể ước lượng batch_size max.")
else:
    torch.cuda.empty_cache()  # giải phóng bộ nhớ
    total_mem = torch.cuda.get_device_properties(device).total_memory
    reserved_mem = torch.cuda.memory_reserved(device)
    allocated_mem = torch.cuda.memory_allocated(device)
    free_mem = total_mem - reserved_mem - allocated_mem
    print(f"Tổng bộ nhớ GPU: {total_mem / 1e9:.2f} GB")
    print(f"Bộ nhớ đã dùng: {allocated_mem / 1e9:.2f} GB")
    print(f"Bộ nhớ trống: {free_mem / 1e9:.2f} GB")
    
    # Giả sử mỗi ảnh 512x512x3 float32 (~3 MB)
    image_size_bytes = 512 * 512 * 3 * 4
    max_batch_size = free_mem // image_size_bytes
    print(f"Batch size tối đa ước lượng: {max_batch_size}")


Tổng bộ nhớ GPU: 33.67 GB
Bộ nhớ đã dùng: 0.00 GB
Bộ nhớ trống: 33.67 GB
Batch size tối đa ước lượng: 10703


# Gen with log

In [6]:
import os
import cv2
import cupy as cp
import numpy as np
import random
import math
import time

class V:
    def __init__(self, x_angle, z_angle, strength):
        self.strength = strength
        self.x_angle = x_angle
        self.z_angle = z_angle
        self.init()
    
    def init(self):
        self.z = self.strength * math.sin(self.z_angle)
        x_y_strength = self.strength * math.cos(self.z_angle)
        self.x = x_y_strength * math.cos(self.x_angle)
        self.y = x_y_strength * math.sin(self.x_angle)
    
    def __str__(self):
        return f"x_angle:{self.x_angle}, z_angle:{self.z_angle}, strength:{self.strength}, x:{self.x}, y:{self.y}, z:{self.z}"

def get_dis_gpu(rows, cols, v, p):
    # Tạo grid tọa độ trên GPU
    row_indices = cp.arange(rows, dtype=cp.float32)
    col_indices = cp.arange(cols, dtype=cp.float32)
    row_grid, col_grid = cp.meshgrid(row_indices, col_indices, indexing='ij')
    
    # Tính toán khoảng cách song song trên GPU
    v_squared = v.x**2 + v.y**2 + v.z**2
    k = (v.x * (col_grid - p[0]) + v.y * (row_grid - p[1])) / v_squared
    
    dis = cp.sqrt((v.x * k + p[0] - col_grid)**2 + 
                  (v.y * k + p[1] - row_grid)**2 + 
                  (v.z * k)**2)
    
    max_dis = float(cp.max(dis))
    return dis.flatten(), max_dis

def curves_folds_image_gpu(new_img_gpu, img_gpu, vec_rows_gpu, vec_cols_gpu, 
                          new_row_list_gpu, new_col_list_gpu,
                          n, curves_folds_num, is_curves, all_num, index, 
                          new_img_row, new_img_col, relax):
    global row_top_move, col_left_move, row_bottom_move, col_right_move, min_row, min_col
    rows, cols = img_gpu.shape[:2]
    max_row = 0
    max_col = 0
    
    for curves_folds_i in range(curves_folds_num):
        random.seed((index + all_num[0]) * int(time.time()) % 2147483647)
        p_row = random.randint(0, new_img_row - 1)
        p_col = random.randint(0, new_img_col - 1)
        alpha = random.random() + 0.1
        v_len = (random.random() + 1) * alpha * 70
        v_x_angle = random.random() * 2 * math.pi
        v_z_angle = 0
        p = (p_col, p_row)
        v = V(v_x_angle, v_z_angle, v_len)
        
        dis_list_gpu, max_dis = get_dis_gpu(rows, cols, v, p)
        
        # Tính toán trọng số song song
        if max_dis > 0:
            dis_normalized = dis_list_gpu / max_dis
        else:
            dis_normalized = cp.zeros_like(dis_list_gpu)
        
        if is_curves:
            w = 1 - cp.power(dis_normalized, alpha)
        else:
            w = alpha / (dis_normalized + alpha)
        
        # Reshape để cập nhật
        w_2d = w.reshape(rows, cols)
        new_row_list_gpu += w_2d * v.y
        new_col_list_gpu += w_2d * v.x
        
        if all_num[0] == n - 1:
            # Tạo grid indices
            row_indices = cp.arange(rows, dtype=cp.int32)
            col_indices = cp.arange(cols, dtype=cp.int32)
            row_grid, col_grid = cp.meshgrid(row_indices, col_indices, indexing='ij')
            
            new_row_grid = (new_row_list_gpu + cp.asarray(row_grid) + relax // 2).astype(cp.int32)
            new_col_grid = (new_col_list_gpu + cp.asarray(col_grid) + relax // 2).astype(cp.int32)
            
            # Tính min/max trên GPU
            min_row = min(min_row, int(cp.min(new_row_grid)))
            max_row = max(max_row, int(cp.max(new_row_grid)))
            min_col = min(min_col, int(cp.min(new_col_grid)))
            max_col = max(max_col, int(cp.max(new_col_grid)))
            
            # Tính toán edge moves
            row_top_move = max(int(cp.max(new_row_grid[0, :])), row_top_move)
            row_bottom_move = min(int(cp.min(new_row_grid[-1, :])), row_bottom_move)
            col_left_move = max(int(cp.max(new_col_grid[:, 0])), col_left_move)
            col_right_move = min(int(cp.min(new_col_grid[:, -1])), col_right_move)
            
            # Lọc valid indices
            valid_mask = ((new_row_grid >= 0) & (new_row_grid < new_img_gpu.shape[0]) &
                         (new_col_grid >= 0) & (new_col_grid < new_img_gpu.shape[1]))
            
            # Gán giá trị cho các vị trí hợp lệ
            valid_rows = new_row_grid[valid_mask]
            valid_cols = new_col_grid[valid_mask]
            source_pixels = img_gpu[valid_mask]
            
            # Cập nhật new_img bằng advanced indexing
            new_img_gpu[valid_rows, valid_cols] = source_pixels
            
            # Cập nhật vectors
            vec_rows_gpu[:] = new_row_grid.flatten()
            vec_cols_gpu[:] = new_col_grid.flatten()
        
        all_num[0] += 1
    
    return max_row, max_col

def folds_curves_img_vector_gpu(img, vec_rows, vec_cols, new_img_row, new_img_col, n, index):
    global row_top_move, col_left_move, row_bottom_move, col_right_move, min_row, min_col
    t_start = time.time()
    
    rows, cols = img.shape[:2]
    relax = 200 * n
    
    # Chuyển sang GPU
    t0 = time.time()
    img_gpu = cp.asarray(img)
    new_img_gpu = cp.ones((rows + relax, cols + relax, 3), dtype=cp.uint8) * cp.array([150, 180, 180], dtype=cp.uint8)
    vec_rows_gpu = cp.asarray(vec_rows)
    vec_cols_gpu = cp.asarray(vec_cols)
    print(f"  [folds_curves] CPU->GPU transfer: {time.time()-t0:.4f}s")
    
    min_row = rows + relax
    min_col = cols + relax
    col_left_move = 0
    row_top_move = 0
    col_right_move = cols + relax + 2
    row_bottom_move = rows + relax + 2
    
    new_row_list_gpu = cp.zeros((rows, cols), dtype=cp.float32)
    new_col_list_gpu = cp.zeros((rows, cols), dtype=cp.float32)
        
    curves_num = random.randint(1, 3)
    folds_num = random.randint(1, 3)
    all_num = [0]
    
    t0 = time.time()
    max_row1, max_col1 = curves_folds_image_gpu(new_img_gpu, img_gpu, vec_rows_gpu, vec_cols_gpu,
                                                new_row_list_gpu, new_col_list_gpu, n, curves_num,
                                                True, all_num, index, new_img_row, new_img_col, relax)
    print(f"  [folds_curves] Curves processing: {time.time()-t0:.4f}s")
    
    t0 = time.time()
    max_row2, max_col2 = curves_folds_image_gpu(new_img_gpu, img_gpu, vec_rows_gpu, vec_cols_gpu,
                                                new_row_list_gpu, new_col_list_gpu, n, folds_num,
                                                False, all_num, index, new_img_row, new_img_col, relax)
    print(f"  [folds_curves] Folds processing: {time.time()-t0:.4f}s")
    
    max_row = max(max_row1, max_row2)
    max_col = max(max_col1, max_col2)
    
    if max_col > min_col and max_row > min_row:
        new_img_gpu = new_img_gpu[min_row:max_row + 1, min_col:max_col + 1]
    
    # Chuyển về CPU
    t0 = time.time()
    vec_rows[:] = cp.asnumpy(vec_rows_gpu)
    vec_cols[:] = cp.asnumpy(vec_cols_gpu)
    
    return cp.asnumpy(new_img_gpu)

import time
import cupy as cp

def linear_interp_gpu(img, field_size,
                           background_color=(150, 180, 180),
                           min_ratio=0.5,
                           verbose=True):
    """
    Thay thế cho phiên bản vòng lặp: vectorized bằng cupyx.scipy.ndimage.convolve.
    - img: numpy array HxWx3 hoặc cupy array
    - field_size: kích thước cửa sổ (nên là số lẻ, vd 3,5,7,...)
    - min_ratio: tỉ lệ tối thiểu các pixel hợp lệ trong cửa sổ để thực hiện trung bình
    """
    t_start = time.time()

    # 1) chuyển ảnh lên GPU (nếu chưa ở đó)
    t0 = time.time()
    img_gpu = cp.asarray(img, dtype=cp.uint8)
    dst_gpu = img_gpu.copy()
    # đồng bộ để đo đúng thời gian chuyển
    cp.cuda.Stream.null.synchronize()
    if verbose:
        print(f"  [linear_interp_fast] CPU->GPU transfer: {time.time()-t0:.4f}s")

    rows, cols = dst_gpu.shape[:2]

    # 2) build mask (True = background pixel cần fill)
    t0 = time.time()
    bg = cp.array(background_color, dtype=cp.uint8)
    mask_bg = cp.all(dst_gpu == bg, axis=2)          # True nơi là background
    valid_mask = (~mask_bg).astype(cp.int32)         # 1 cho pixel hợp lệ (không phải nền)
    cp.cuda.Stream.null.synchronize()
    if verbose:
        print(f"  [linear_interp_fast] Create mask: {time.time()-t0:.4f}s")

    # 3) convolution để tính count và tổng màu trong cửa sổ
    t0 = time.time()
    try:
        # cupyx.scipy.ndimage.convolve thường có sẵn trong Cupy
        from cupyx.scipy.ndimage import convolve
    except Exception:
        raise RuntimeError("cupyx.scipy.ndimage.convolve is required (install cupy with cupyx).")

    kernel = cp.ones((field_size, field_size), dtype=cp.int32)

    # đếm pixel hợp lệ trong mỗi cửa sổ
    count = convolve(valid_mask, kernel, mode='constant', cval=0)

    # tổng từng kênh (dùng int32 để tránh tràn)
    sums = cp.empty((rows, cols, 3), dtype=cp.int32)
    for c in range(3):
        ch = dst_gpu[:, :, c].astype(cp.int32) * valid_mask
        sums[:, :, c] = convolve(ch, kernel, mode='constant', cval=0)

    # đồng bộ trước khi đo thời gian
    cp.cuda.Stream.null.synchronize()
    if verbose:
        print(f"  [linear_interp_fast] Convolutions: {time.time()-t0:.4f}s")

    # 4) tính mean và gán vào những pixel cần fill thỏa điều kiện
    t0 = time.time()
    min_count = int(field_size * field_size * min_ratio)
    fill_pos = mask_bg & (count >= min_count)   # chỉ fill background và có đủ pixel hợp lệ

    # tránh chia cho 0
    count_safe = cp.where(count == 0, 1, count).astype(cp.float32)
    mean = (sums.astype(cp.float32) / count_safe[..., None]).astype(cp.uint8)

    # gán màu trung bình cho các vị trí cần fill
    dst_gpu[fill_pos] = mean[fill_pos]

    # đồng bộ và đo thời gian
    cp.cuda.Stream.null.synchronize()
    if verbose:
        print(f"  [linear_interp_fast] Fill & finalize: {time.time()-t0:.4f}s")
        print(f"  [linear_interp_fast] TOTAL: {time.time()-t_start:.4f}s")

    # chuyển về CPU (nếu cần)
    return cp.asnumpy(dst_gpu)


def linear_interp_gpu_2(img, field_size):
    t_start = time.time()
    
    # Transfer to GPU
    t0 = time.time()
    img_gpu = cp.asarray(img, dtype=cp.uint8)
    dst_image_gpu = img_gpu.copy()
    print(f"  [linear_interp] CPU->GPU transfer: {time.time()-t0:.4f}s")
    
    rows, cols = img_gpu.shape[:2]
    start = -field_size // 2
    end = field_size // 2 + 1
    
    # Create mask
    t0 = time.time()
    background_color = cp.array([150, 180, 180], dtype=cp.uint8)
    mask = cp.all(img_gpu == background_color, axis=2)
    print(f"  [linear_interp] Create mask: {time.time()-t0:.4f}s")
    
    # Vectorized processing
    t0 = time.time()
    
    # Tìm tọa độ các pixel cần xử lý
    bg_rows, bg_cols = cp.where(mask[abs(start):rows-end, abs(start):cols-end])
    bg_rows += abs(start)
    bg_cols += abs(start)
    count = len(bg_rows)
    
    if count > 0:
        # Tạo offset grid cho neighborhood
        offsets = cp.arange(start, end)
        row_offsets, col_offsets = cp.meshgrid(offsets, offsets, indexing='ij')
        row_offsets = row_offsets.ravel()
        col_offsets = col_offsets.ravel()
        
        # Broadcast để lấy tất cả neighbors cùng lúc
        neighbor_rows = bg_rows[:, None] + row_offsets[None, :]  # (count, field_size²)
        neighbor_cols = bg_cols[:, None] + col_offsets[None, :]
        
        # Lấy giá trị neighbors
        neighbor_pixels = img_gpu[neighbor_rows, neighbor_cols]  # (count, field_size², 3)
        
        # Kiểm tra valid pixels (không phải background)
        is_valid = ~cp.all(neighbor_pixels == background_color, axis=2)  # (count, field_size²)
        
        # Đếm valid pixels cho mỗi vị trí
        valid_count = cp.sum(is_valid, axis=1)  # (count,)
        threshold = field_size * field_size // 2
        enough_valid = valid_count > threshold
        
        # Tính mean chỉ cho pixels có đủ valid neighbors
        if cp.any(enough_valid):
            # Mask out invalid pixels
            neighbor_pixels_float = neighbor_pixels.astype(cp.float32)
            neighbor_pixels_float[~is_valid] = cp.nan
            
            # Compute mean (ignoring NaN)
            mean_colors = cp.nanmean(neighbor_pixels_float, axis=1)  # (count, 3)
            mean_colors = cp.round(mean_colors).astype(cp.uint8)
            
            # Update only positions with enough valid neighbors
            update_rows = bg_rows[enough_valid]
            update_cols = bg_cols[enough_valid]
            dst_image_gpu[update_rows, update_cols] = mean_colors[enough_valid]
    
    print(f"  [linear_interp] Vectorized processing ({count} pixels): {time.time()-t0:.4f}s")
    
    # Transfer back
    t0 = time.time()
    result = cp.asnumpy(dst_image_gpu)
    print(f"  [linear_interp] GPU->CPU transfer: {time.time()-t0:.4f}s")
    print(f"  [linear_interp] TOTAL: {time.time()-t_start:.4f}s")
    
    return result

def warp_images(new_img, img, vec_rows, vec_cols, index):
    global row_top_move, col_left_move, row_bottom_move, col_right_move, min_row, min_col
    t_start = time.time()
    
    random.seed(index * int(time.time()) % 2147483647)
    rows, cols = new_img.shape[:2]
    
    p1 = np.float32([0, 0])
    p2 = np.float32([cols - 1, 0])
    p3 = np.float32([0, rows - 1])
    p4 = np.float32([cols - 1, rows - 1])
    src_points = np.array([p1, p2, p3, p4])
    
    p1_dx = random.randint(0, 5)
    p1_dy = random.randint(0, 5)
    p1[0] += p1_dx
    p1[1] += p1_dy
    p2[0] -= p1_dx
    p2[1] += p1_dy
    
    p3_dx = random.randint(0, 3)
    p3_dy = random.randint(0, 3)
    p3[0] += p3_dx
    p3[1] -= p3_dy
    p4[0] -= p3_dx
    p4[1] -= p3_dy
    # Kiểm tra ảnh đầu vào
    if new_img is None or new_img.size == 0:
        raise ValueError(f"[warp_images] Empty new_img at index {index}")
    rows, cols = new_img.shape[:2]
    if rows == 0 or cols == 0:
        raise ValueError(f"[warp_images] Invalid shape {new_img.shape} at index {index}")

    # Kiểm tra input cho getPerspectiveTransform
    if vec_rows.shape[0] < 4 or vec_cols.shape[0] < 4:
        raise ValueError(f"[warp_images] Not enough points: rows={vec_rows.shape}, cols={vec_cols.shape}")
    if np.isnan(vec_rows).any() or np.isnan(vec_cols).any():
        raise ValueError(f"[warp_images] NaN in transform points at index {index}")

    dst_points = np.array([p1, p2, p3, p4])
    M = cv2.getPerspectiveTransform(src_points, dst_points)
    
    t0 = time.time()
    new_img = cv2.warpPerspective(new_img, M, (cols, rows), flags=cv2.INTER_LINEAR,
                                 borderMode=cv2.BORDER_CONSTANT,
                                 borderValue=(150, 180, 180))
    print(f"  [warp] warpPerspective: {time.time()-t0:.4f}s")
    
    img_rows, img_cols = img.shape[:2]
    
    # Chuyển sang GPU cho phép biến đổi perspective
    t0 = time.time()
    vec_rows_gpu = cp.asarray(vec_rows).reshape(-1)
    vec_cols_gpu = cp.asarray(vec_cols).reshape(-1)
    
    points = cp.stack([vec_cols_gpu - min_col, vec_rows_gpu - min_row], axis=1)
    points_np = cp.asnumpy(points).astype(np.float32).reshape(-1, 1, 2)
    
    points_trans = cv2.perspectiveTransform(points_np, M)
    points_trans_gpu = cp.asarray(points_trans).reshape(-1, 2)
    
    new_row_grid = points_trans_gpu[:, 1].astype(cp.int32).reshape(img_rows, img_cols)
    new_col_grid = points_trans_gpu[:, 0].astype(cp.int32).reshape(img_rows, img_cols)

    
    vec_rows[:] = cp.asnumpy(new_row_grid.flatten())
    vec_cols[:] = cp.asnumpy(new_col_grid.flatten())
    
    min_row = int(cp.min(new_row_grid))
    min_col = int(cp.min(new_col_grid))
    
    row_top_move = int(cp.max(new_row_grid[0, :]))
    row_bottom_move = int(cp.min(new_row_grid[-1, :]))
    col_left_move = int(cp.max(new_col_grid[:, 0]))
    col_right_move = int(cp.min(new_col_grid[:, -1]))
    print(f"  [warp] Point transform: {time.time()-t0:.4f}s")
    print(f"  [warp] TOTAL: {time.time()-t_start:.4f}s")
    
    return new_img


    
def safe_imread(path, index=None):
    img = cv2.imread(path)
    if img is None or img.size == 0:
        raise ValueError(f"[safe_imread] Failed to read image at {path}, index {index}")
    return img

def add_background(img, background_path, board):
    global row_top_move, col_left_move, row_bottom_move, col_right_move
    t_start = time.time()
    
    rows, cols = img.shape[:2]
    t0 = time.time()
    background_image = safe_imread(background_path)
    background_image = cv2.resize(background_image, (cols + board * 2, rows + board * 2))
    print(f"  [add_bg] Load & resize bg: {time.time()-t0:.4f}s")
    
    # Chuyển sang GPU
    t0 = time.time()
    img_gpu = cp.asarray(img)
    bg_gpu = cp.asarray(background_image)
    print(f"  [add_bg] CPU->GPU transfer: {time.time()-t0:.4f}s")
    
    row_start = board
    col_start = board
    
    background_color = cp.array([150, 180, 180], dtype=cp.uint8)
    
    # Tạo mask cho vùng inner và outer
    t0 = time.time()
    row_indices = cp.arange(rows)
    col_indices = cp.arange(cols)
    row_grid, col_grid = cp.meshgrid(row_indices, col_indices, indexing='ij')
    
    inner_mask = ((col_grid > col_left_move + 10) & (col_grid < col_right_move - 10) &
                  (row_grid > row_top_move + 10) & (row_grid < row_bottom_move - 10))
    
    not_background = ~cp.all(img_gpu == background_color, axis=2)
    outer_mask = not_background & ~inner_mask
    
    # Gán giá trị
    bg_gpu[row_start:row_start + rows, col_start:col_start + cols][inner_mask] = img_gpu[inner_mask]
    bg_gpu[row_start:row_start + rows, col_start:col_start + cols][outer_mask] = img_gpu[outer_mask]
    print(f"  [add_bg] Mask & blend: {time.time()-t0:.4f}s")
    
    t0 = time.time()
    result = cp.asnumpy(bg_gpu)
    print(f"  [add_bg] GPU->CPU transfer: {time.time()-t0:.4f}s")
    print(f"  [add_bg] TOTAL: {time.time()-t_start:.4f}s")
    
    return cp.asnumpy(bg_gpu)

def generate_one_img_py(img_path, background_path, save_path, new_row, new_col,
                       vec_rows, vec_cols, n, index, kernel):
    global row_top_move, col_left_move, row_bottom_move, col_right_move, min_row, min_col
    
    print("\n" + "="*60)
    print(f"Processing image: {os.path.basename(img_path)}")
    print("="*60)
    
    t_total = time.time()
    
    t0 = time.time()
    src = safe_imread(img_path)
    if src is None:
        return 0
    src = cv2.resize(src, (new_col, new_row))
    print(f"[MAIN] Load & resize image: {time.time()-t0:.4f}s")
    
    print("\n--- Step 1: Folds & Curves ---")
    new_img = folds_curves_img_vector_gpu(src, vec_rows, vec_cols, new_row, new_col, n, index)
    
    print("\n--- Step 2: Linear Interpolation ---")
    dst_img = linear_interp_gpu(new_img, kernel)
    
    print("\n--- Step 3: Warp Images ---")
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
    
    print("\n--- Step 4: Add Background ---")
    board = 0
    final_img = add_background(dst_img, background_path, board)
    
    t0 = time.time()
    rows, cols = src.shape[:2]
    ratio_row = rows / final_img.shape[0]
    ratio_col = cols / final_img.shape[1]
    
    vec_rows[:] = (vec_rows + board) * ratio_row
    vec_cols[:] = (vec_cols + board) * ratio_col
    
    final_img = cv2.resize(final_img, (cols, rows))
    cv2.imwrite(save_path, final_img)
    print(f"[MAIN] Final resize & save: {time.time()-t0:.4f}s")
    
    print(f"\n{'='*60}")
    print(f"TOTAL PROCESSING TIME: {time.time()-t_total:.4f}s")
    print(f"{'='*60}\n")
    
    return 1

def generate_a_sample(img_path, background_path, save_path, i=0):
    try:
        max_new_row = 512
        max_new_col = 512
        img = cv2.imread(img_path)
        if img is None:
            return
        new_col = max_new_col
        new_row = max_new_row
        map_rows = np.ones((new_row * new_col,), dtype=np.float32)
        map_cols = np.ones((new_row * new_col,), dtype=np.float32)
        np.random.seed(i)
        kernel = 3
        n = 1
        result = generate_one_img_py(
            img_path, background_path, save_path,
            new_row, new_col, map_rows, map_cols,
            n, i, kernel
        )
        if result == 1:
            map_rows_2d = map_rows.reshape(new_row, new_col)
            map_cols_2d = map_cols.reshape(new_row, new_col)
            mapping_data = np.stack((map_cols_2d, map_rows_2d), axis=2)
            label_dir = os.path.join(os.path.dirname(save_path).replace("images", "labels"))
            os.makedirs(label_dir, exist_ok=True)
            np.save(os.path.join(label_dir, f"{os.path.splitext(os.path.basename(save_path))[0]}.npy"), mapping_data)
    except Exception as e:
        import traceback
        traceback.print_exc()

def batch_process():
    input_dir = "/workspace/scan_images"
    background_dir = "/workspace/backgrounds"
    output_dir = "test_gen"
    start_idx = 0000
    max_images = 1300
    os.makedirs(output_dir, exist_ok=True)
    input_images = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))] * 20

    print(f"Found {len(input_images)} input images.")
    background_images = []
    for root, dirs, files in os.walk(background_dir):
        for file in files:
            if os.path.splitext(file)[1].lower() in {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}:
                background_images.append(os.path.join(root, file))
    if not input_images:
        return
    if not background_images:
        return
    if max_images:
        input_images = input_images[:max_images]
    
    batch_start = time.time()
    for i, img_name in enumerate(input_images):
        current_idx = start_idx + i
        bg_idx = current_idx % len(background_images)
        img_path = os.path.join(input_dir, img_name)
        background_path = background_images[bg_idx]
        name_without_ext = os.path.splitext(img_name)[0]
        save_dir = os.path.join(output_dir, "images")
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"{current_idx:06d}.jpg")
        generate_a_sample(img_path, background_path, save_path, current_idx)
    
    print(f"\n{'#'*60}")
    print(f"BATCH COMPLETE - Total time: {time.time()-batch_start:.2f}s")
    print(f"Average per image: {(time.time()-batch_start)/len(input_images):.2f}s")
    print(f"{'#'*60}")

if __name__ == "__main__":
    batch_process()

Found 15460 input images.

Processing image: X00016469612.jpg
[MAIN] Load & resize image: 0.0019s

--- Step 1: Folds & Curves ---
  [folds_curves] CPU->GPU transfer: 0.0084s
  [folds_curves] Curves processing: 0.0023s
  [folds_curves] Folds processing: 0.0006s

--- Step 2: Linear Interpolation ---
  [linear_interp_fast] CPU->GPU transfer: 0.0001s
  [linear_interp_fast] Create mask: 0.0002s
  [linear_interp_fast] Convolutions: 0.0005s
  [linear_interp_fast] Fill & finalize: 0.0005s
  [linear_interp_fast] TOTAL: 0.0015s

--- Step 3: Warp Images ---
  [warp] warpPerspective: 0.0017s
  [warp] Point transform: 0.0019s
  [warp] TOTAL: 0.0037s

--- Step 4: Add Background ---
  [add_bg] Load & resize bg: 0.0006s
  [add_bg] CPU->GPU transfer: 0.0002s
  [add_bg] Mask & blend: 0.2521s
  [add_bg] GPU->CPU transfer: 0.0004s
  [add_bg] TOTAL: 0.2535s
[MAIN] Final resize & save: 0.0017s

TOTAL PROCESSING TIME: 0.2743s


Processing image: X00016469619.jpg
[MAIN] Load & resize image: 0.0018s

--- Step 

Traceback (most recent call last):
  File "/tmp/ipykernel_4304/2187578760.py", line 526, in generate_a_sample
    result = generate_one_img_py(
             ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/2187578760.py", line 488, in generate_one_img_py
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/2187578760.py", line 359, in warp_images
    raise ValueError(f"[warp_images] Empty new_img at index {index}")
ValueError: [warp_images] Empty new_img at index 92
Traceback (most recent call last):
  File "/tmp/ipykernel_4304/2187578760.py", line 526, in generate_a_sample
    result = generate_one_img_py(
             ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/2187578760.py", line 488, in generate_one_img_py
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/218

  [warp] Point transform: 0.0023s
  [warp] TOTAL: 0.0038s

--- Step 4: Add Background ---
  [add_bg] Load & resize bg: 0.0013s
  [add_bg] CPU->GPU transfer: 0.0002s
  [add_bg] Mask & blend: 0.0011s
  [add_bg] GPU->CPU transfer: 0.0001s
  [add_bg] TOTAL: 0.0028s
[MAIN] Final resize & save: 0.0016s

TOTAL PROCESSING TIME: 0.0197s


Processing image: X51005587254.jpg
[MAIN] Load & resize image: 0.0136s

--- Step 1: Folds & Curves ---
  [folds_curves] CPU->GPU transfer: 0.0006s
  [folds_curves] Curves processing: 0.0029s
  [folds_curves] Folds processing: 0.0010s

--- Step 2: Linear Interpolation ---
  [linear_interp_fast] CPU->GPU transfer: 0.0001s
  [linear_interp_fast] Create mask: 0.0003s
  [linear_interp_fast] Convolutions: 0.0005s
  [linear_interp_fast] Fill & finalize: 0.0005s
  [linear_interp_fast] TOTAL: 0.0015s

--- Step 3: Warp Images ---
  [warp] warpPerspective: 0.0012s
  [warp] Point transform: 0.0018s
  [warp] TOTAL: 0.0031s

--- Step 4: Add Background ---
  [add_bg] Load & 

Traceback (most recent call last):
  File "/tmp/ipykernel_4304/2187578760.py", line 526, in generate_a_sample
    result = generate_one_img_py(
             ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/2187578760.py", line 488, in generate_one_img_py
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/2187578760.py", line 359, in warp_images
    raise ValueError(f"[warp_images] Empty new_img at index {index}")
ValueError: [warp_images] Empty new_img at index 132


  [folds_curves] CPU->GPU transfer: 0.0009s
  [folds_curves] Curves processing: 0.0023s
  [folds_curves] Folds processing: 0.0005s

--- Step 2: Linear Interpolation ---
  [linear_interp_fast] CPU->GPU transfer: 0.0001s
  [linear_interp_fast] Create mask: 0.0002s
  [linear_interp_fast] Convolutions: 0.0005s
  [linear_interp_fast] Fill & finalize: 0.0005s
  [linear_interp_fast] TOTAL: 0.0015s

--- Step 3: Warp Images ---
  [warp] warpPerspective: 0.0014s
  [warp] Point transform: 0.0018s
  [warp] TOTAL: 0.0034s

--- Step 4: Add Background ---
  [add_bg] Load & resize bg: 0.0052s
  [add_bg] CPU->GPU transfer: 0.0002s
  [add_bg] Mask & blend: 0.0011s
  [add_bg] GPU->CPU transfer: 0.0001s
  [add_bg] TOTAL: 0.0066s
[MAIN] Final resize & save: 0.0016s

TOTAL PROCESSING TIME: 0.0233s


Processing image: X51005676540.jpg
[MAIN] Load & resize image: 0.0073s

--- Step 1: Folds & Curves ---
  [folds_curves] CPU->GPU transfer: 0.0004s
  [folds_curves] Curves processing: 0.0027s
  [folds_curves] Fol

...

Traceback (most recent call last):
  File "/tmp/ipykernel_4304/2187578760.py", line 526, in generate_a_sample
    result = generate_one_img_py(
             ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/2187578760.py", line 488, in generate_one_img_py
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/2187578760.py", line 359, in warp_images
    raise ValueError(f"[warp_images] Empty new_img at index {index}")
ValueError: [warp_images] Empty new_img at index 1258



Processing image: X51006557188.jpg
[MAIN] Load & resize image: 0.0046s

--- Step 1: Folds & Curves ---
  [folds_curves] CPU->GPU transfer: 0.0004s
  [folds_curves] Curves processing: 0.0017s
  [folds_curves] Folds processing: 0.0005s

--- Step 2: Linear Interpolation ---
  [linear_interp_fast] CPU->GPU transfer: 0.0001s
  [linear_interp_fast] Create mask: 0.0002s
  [linear_interp_fast] Convolutions: 0.0005s
  [linear_interp_fast] Fill & finalize: 0.0005s
  [linear_interp_fast] TOTAL: 0.0014s

--- Step 3: Warp Images ---
  [warp] warpPerspective: 0.0014s
  [warp] Point transform: 0.0026s
  [warp] TOTAL: 0.0041s

--- Step 4: Add Background ---
  [add_bg] Load & resize bg: 0.0019s
  [add_bg] CPU->GPU transfer: 0.0002s
  [add_bg] Mask & blend: 0.0010s
  [add_bg] GPU->CPU transfer: 0.0001s
  [add_bg] TOTAL: 0.0033s
[MAIN] Final resize & save: 0.0016s

TOTAL PROCESSING TIME: 0.0184s


Processing image: X51006557193.jpg
[MAIN] Load & resize image: 0.0046s

--- Step 1: Folds & Curves ---
  [f

Traceback (most recent call last):
  File "/tmp/ipykernel_4304/2187578760.py", line 526, in generate_a_sample
    result = generate_one_img_py(
             ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/2187578760.py", line 488, in generate_one_img_py
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/2187578760.py", line 359, in warp_images
    raise ValueError(f"[warp_images] Empty new_img at index {index}")
ValueError: [warp_images] Empty new_img at index 1268



--- Step 3: Warp Images ---
  [warp] warpPerspective: 0.0012s
  [warp] Point transform: 0.0026s
  [warp] TOTAL: 0.0044s

--- Step 4: Add Background ---
  [add_bg] Load & resize bg: 0.0008s
  [add_bg] CPU->GPU transfer: 0.0002s
  [add_bg] Mask & blend: 0.0011s
  [add_bg] GPU->CPU transfer: 0.0001s
  [add_bg] TOTAL: 0.0022s
[MAIN] Final resize & save: 0.0019s

TOTAL PROCESSING TIME: 0.0195s


Processing image: X51006557209.jpg
[MAIN] Load & resize image: 0.0040s

--- Step 1: Folds & Curves ---
  [folds_curves] CPU->GPU transfer: 0.0005s
  [folds_curves] Curves processing: 0.0018s
  [folds_curves] Folds processing: 0.0006s

--- Step 2: Linear Interpolation ---
  [linear_interp_fast] CPU->GPU transfer: 0.0002s
  [linear_interp_fast] Create mask: 0.0003s
  [linear_interp_fast] Convolutions: 0.0005s
  [linear_interp_fast] Fill & finalize: 0.0006s
  [linear_interp_fast] TOTAL: 0.0016s

--- Step 3: Warp Images ---
  [warp] warpPerspective: 0.0013s
  [warp] Point transform: 0.0022s
  [warp] TO

Traceback (most recent call last):
  File "/tmp/ipykernel_4304/2187578760.py", line 526, in generate_a_sample
    result = generate_one_img_py(
             ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/2187578760.py", line 488, in generate_one_img_py
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/2187578760.py", line 359, in warp_images
    raise ValueError(f"[warp_images] Empty new_img at index {index}")
ValueError: [warp_images] Empty new_img at index 1292


# Gen without log

In [18]:
import os
import cv2
import cupy as cp
import numpy as np
import random
import math
import time

class V:
    def __init__(self, x_angle, z_angle, strength):
        self.strength = strength
        self.x_angle = x_angle
        self.z_angle = z_angle
        self.init()
    
    def init(self):
        self.z = self.strength * math.sin(self.z_angle)
        x_y_strength = self.strength * math.cos(self.z_angle)
        self.x = x_y_strength * math.cos(self.x_angle)
        self.y = x_y_strength * math.sin(self.x_angle)
    
    def __str__(self):
        return f"x_angle:{self.x_angle}, z_angle:{self.z_angle}, strength:{self.strength}, x:{self.x}, y:{self.y}, z:{self.z}"

def get_dis_gpu(rows, cols, v, p):
    row_indices = cp.arange(rows, dtype=cp.float32)
    col_indices = cp.arange(cols, dtype=cp.float32)
    row_grid, col_grid = cp.meshgrid(row_indices, col_indices, indexing='ij')
    
    v_squared = v.x**2 + v.y**2 + v.z**2
    k = (v.x * (col_grid - p[0]) + v.y * (row_grid - p[1])) / v_squared
    
    dis = cp.sqrt((v.x * k + p[0] - col_grid)**2 + 
                  (v.y * k + p[1] - row_grid)**2 + 
                  (v.z * k)**2)
    
    max_dis = float(cp.max(dis))
    return dis.flatten(), max_dis

def curves_folds_image_gpu(new_img_gpu, img_gpu, vec_rows_gpu, vec_cols_gpu, 
                          new_row_list_gpu, new_col_list_gpu,
                          n, curves_folds_num, is_curves, all_num, index, 
                          new_img_row, new_img_col, relax):
    global row_top_move, col_left_move, row_bottom_move, col_right_move, min_row, min_col
    rows, cols = img_gpu.shape[:2]
    max_row = 0
    max_col = 0
    
    for curves_folds_i in range(curves_folds_num):
        random.seed((index + all_num[0]) * int(time.time()) % 2147483647)
        p_row = random.randint(0, new_img_row - 1)
        p_col = random.randint(0, new_img_col - 1)
        alpha = random.random() + 0.1
        v_len = (random.random() + 1) * alpha * 70
        v_x_angle = random.random() * 2 * math.pi
        v_z_angle = 0
        p = (p_col, p_row)
        v = V(v_x_angle, v_z_angle, v_len)
        
        dis_list_gpu, max_dis = get_dis_gpu(rows, cols, v, p)
        
        if max_dis > 0:
            dis_normalized = dis_list_gpu / max_dis
        else:
            dis_normalized = cp.zeros_like(dis_list_gpu)
        
        if is_curves:
            w = 1 - cp.power(dis_normalized, alpha)
        else:
            w = alpha / (dis_normalized + alpha)
        
        w_2d = w.reshape(rows, cols)
        new_row_list_gpu += w_2d * v.y
        new_col_list_gpu += w_2d * v.x
        
        if all_num[0] == n - 1:
            row_indices = cp.arange(rows, dtype=cp.int32)
            col_indices = cp.arange(cols, dtype=cp.int32)
            row_grid, col_grid = cp.meshgrid(row_indices, col_indices, indexing='ij')
            
            new_row_grid = (new_row_list_gpu + cp.asarray(row_grid) + relax // 2).astype(cp.int32)
            new_col_grid = (new_col_list_gpu + cp.asarray(col_grid) + relax // 2).astype(cp.int32)
            
            min_row = min(min_row, int(cp.min(new_row_grid)))
            max_row = max(max_row, int(cp.max(new_row_grid)))
            min_col = min(min_col, int(cp.min(new_col_grid)))
            max_col = max(max_col, int(cp.max(new_col_grid)))
            
            row_top_move = max(int(cp.max(new_row_grid[0, :])), row_top_move)
            row_bottom_move = min(int(cp.min(new_row_grid[-1, :])), row_bottom_move)
            col_left_move = max(int(cp.max(new_col_grid[:, 0])), col_left_move)
            col_right_move = min(int(cp.min(new_col_grid[:, -1])), col_right_move)
            
            valid_mask = ((new_row_grid >= 0) & (new_row_grid < new_img_gpu.shape[0]) &
                         (new_col_grid >= 0) & (new_col_grid < new_img_gpu.shape[1]))
            
            valid_rows = new_row_grid[valid_mask]
            valid_cols = new_col_grid[valid_mask]
            source_pixels = img_gpu[valid_mask]
            
            new_img_gpu[valid_rows, valid_cols] = source_pixels
            
            vec_rows_gpu[:] = new_row_grid.flatten()
            vec_cols_gpu[:] = new_col_grid.flatten()
        
        all_num[0] += 1
    
    return max_row, max_col

def folds_curves_img_vector_gpu(img, vec_rows, vec_cols, new_img_row, new_img_col, n, index):
    global row_top_move, col_left_move, row_bottom_move, col_right_move, min_row, min_col
    
    rows, cols = img.shape[:2]
    relax = 200 * n
    
    img_gpu = cp.asarray(img)
    new_img_gpu = cp.ones((rows + relax, cols + relax, 3), dtype=cp.uint8) * cp.array([150, 180, 180], dtype=cp.uint8)
    vec_rows_gpu = cp.asarray(vec_rows)
    vec_cols_gpu = cp.asarray(vec_cols)
    
    min_row = rows + relax
    min_col = cols + relax
    col_left_move = 0
    row_top_move = 0
    col_right_move = cols + relax + 2
    row_bottom_move = rows + relax + 2
    
    new_row_list_gpu = cp.zeros((rows, cols), dtype=cp.float32)
    new_col_list_gpu = cp.zeros((rows, cols), dtype=cp.float32)
        
    curves_num = random.randint(1, 5)
    folds_num = random.randint(1, 5)
    all_num = [0]
    
    max_row1, max_col1 = curves_folds_image_gpu(new_img_gpu, img_gpu, vec_rows_gpu, vec_cols_gpu,
                                                new_row_list_gpu, new_col_list_gpu, n, curves_num,
                                                True, all_num, index, new_img_row, new_img_col, relax)
    
    max_row2, max_col2 = curves_folds_image_gpu(new_img_gpu, img_gpu, vec_rows_gpu, vec_cols_gpu,
                                                new_row_list_gpu, new_col_list_gpu, n, folds_num,
                                                False, all_num, index, new_img_row, new_img_col, relax)
    
    max_row = max(max_row1, max_row2)
    max_col = max(max_col1, max_col2)
    
    if max_col > min_col and max_row > min_row:
        new_img_gpu = new_img_gpu[min_row:max_row + 1, min_col:max_col + 1]
    
    vec_rows[:] = cp.asnumpy(vec_rows_gpu)
    vec_cols[:] = cp.asnumpy(vec_cols_gpu)
    
    return cp.asnumpy(new_img_gpu)

def linear_interp_gpu(img, field_size,
                     background_color=(150, 180, 180),
                     min_ratio=0.5,
                     verbose=False):
    img_gpu = cp.asarray(img, dtype=cp.uint8)
    dst_gpu = img_gpu.copy()
    cp.cuda.Stream.null.synchronize()

    rows, cols = dst_gpu.shape[:2]

    bg = cp.array(background_color, dtype=cp.uint8)
    mask_bg = cp.all(dst_gpu == bg, axis=2)
    valid_mask = (~mask_bg).astype(cp.int32)
    cp.cuda.Stream.null.synchronize()

    try:
        from cupyx.scipy.ndimage import convolve
    except Exception:
        raise RuntimeError("cupyx.scipy.ndimage.convolve is required (install cupy with cupyx).")

    kernel = cp.ones((field_size, field_size), dtype=cp.int32)

    count = convolve(valid_mask, kernel, mode='constant', cval=0)

    sums = cp.empty((rows, cols, 3), dtype=cp.int32)
    for c in range(3):
        ch = dst_gpu[:, :, c].astype(cp.int32) * valid_mask
        sums[:, :, c] = convolve(ch, kernel, mode='constant', cval=0)

    cp.cuda.Stream.null.synchronize()

    min_count = int(field_size * field_size * min_ratio)
    fill_pos = mask_bg & (count >= min_count)

    count_safe = cp.where(count == 0, 1, count).astype(cp.float32)
    mean = (sums.astype(cp.float32) / count_safe[..., None]).astype(cp.uint8)

    dst_gpu[fill_pos] = mean[fill_pos]

    cp.cuda.Stream.null.synchronize()

    return cp.asnumpy(dst_gpu)

def warp_images(new_img, img, vec_rows, vec_cols, index):
    global row_top_move, col_left_move, row_bottom_move, col_right_move, min_row, min_col
    
    random.seed(index * int(time.time()) % 2147483647)
    rows, cols = new_img.shape[:2]
    
    p1 = np.float32([0, 0])
    p2 = np.float32([cols - 1, 0])
    p3 = np.float32([0, rows - 1])
    p4 = np.float32([cols - 1, rows - 1])
    src_points = np.array([p1, p2, p3, p4])
    
    p1_dx = random.randint(0, 5)
    p1_dy = random.randint(0, 5)
    p1[0] += p1_dx
    p1[1] += p1_dy
    p2[0] -= p1_dx
    p2[1] += p1_dy
    
    p3_dx = random.randint(0, 3)
    p3_dy = random.randint(0, 3)
    p3[0] += p3_dx
    p3[1] -= p3_dy
    p4[0] -= p3_dx
    p4[1] -= p3_dy
    
    if new_img is None or new_img.size == 0:
        raise ValueError(f"[warp_images] Empty new_img at index {index}")
    rows, cols = new_img.shape[:2]
    if rows == 0 or cols == 0:
        raise ValueError(f"[warp_images] Invalid shape {new_img.shape} at index {index}")

    if vec_rows.shape[0] < 4 or vec_cols.shape[0] < 4:
        raise ValueError(f"[warp_images] Not enough points: rows={vec_rows.shape}, cols={vec_cols.shape}")
    if np.isnan(vec_rows).any() or np.isnan(vec_cols).any():
        raise ValueError(f"[warp_images] NaN in transform points at index {index}")

    dst_points = np.array([p1, p2, p3, p4])
    M = cv2.getPerspectiveTransform(src_points, dst_points)
    
    new_img = cv2.warpPerspective(new_img, M, (cols, rows), flags=cv2.INTER_LINEAR,
                                 borderMode=cv2.BORDER_CONSTANT,
                                 borderValue=(150, 180, 180))
    
    img_rows, img_cols = img.shape[:2]
    
    vec_rows_gpu = cp.asarray(vec_rows).reshape(-1)
    vec_cols_gpu = cp.asarray(vec_cols).reshape(-1)
    
    points = cp.stack([vec_cols_gpu - min_col, vec_rows_gpu - min_row], axis=1)
    points_np = cp.asnumpy(points).astype(np.float32).reshape(-1, 1, 2)
    
    points_trans = cv2.perspectiveTransform(points_np, M)
    points_trans_gpu = cp.asarray(points_trans).reshape(-1, 2)
    
    new_row_grid = points_trans_gpu[:, 1].astype(cp.int32).reshape(img_rows, img_cols)
    new_col_grid = points_trans_gpu[:, 0].astype(cp.int32).reshape(img_rows, img_cols)
    
    vec_rows[:] = cp.asnumpy(new_row_grid.flatten())
    vec_cols[:] = cp.asnumpy(new_col_grid.flatten())
    
    min_row = int(cp.min(new_row_grid))
    min_col = int(cp.min(new_col_grid))
    
    row_top_move = int(cp.max(new_row_grid[0, :]))
    row_bottom_move = int(cp.min(new_row_grid[-1, :]))
    col_left_move = int(cp.max(new_col_grid[:, 0]))
    col_right_move = int(cp.min(new_col_grid[:, -1]))
    
    return new_img

def safe_imread(path, index=None):
    img = cv2.imread(path)
    if img is None or img.size == 0:
        raise ValueError(f"[safe_imread] Failed to read image at {path}, index {index}")
    return img

def add_background(img, background_path, board):
    global row_top_move, col_left_move, row_bottom_move, col_right_move
    
    rows, cols = img.shape[:2]
    background_image = safe_imread(background_path)
    background_image = cv2.resize(background_image, (cols + board * 2, rows + board * 2))
    
    img_gpu = cp.asarray(img)
    bg_gpu = cp.asarray(background_image)
    
    row_start = board
    col_start = board
    
    background_color = cp.array([150, 180, 180], dtype=cp.uint8)
    
    row_indices = cp.arange(rows)
    col_indices = cp.arange(cols)
    row_grid, col_grid = cp.meshgrid(row_indices, col_indices, indexing='ij')
    
    inner_mask = ((col_grid > col_left_move + 10) & (col_grid < col_right_move - 10) &
                  (row_grid > row_top_move + 10) & (row_grid < row_bottom_move - 10))
    
    not_background = ~cp.all(img_gpu == background_color, axis=2)
    outer_mask = not_background & ~inner_mask
    
    bg_gpu[row_start:row_start + rows, col_start:col_start + cols][inner_mask] = img_gpu[inner_mask]
    bg_gpu[row_start:row_start + rows, col_start:col_start + cols][outer_mask] = img_gpu[outer_mask]
    
    return cp.asnumpy(bg_gpu)

def generate_one_img_py(img_path, background_path, save_path, new_row, new_col,
                       vec_rows, vec_cols, n, index, kernel):
    global row_top_move, col_left_move, row_bottom_move, col_right_move, min_row, min_col
    
    src = safe_imread(img_path)
    if src is None:
        return 0
    src = cv2.resize(src, (new_col, new_row))
    
    new_img = folds_curves_img_vector_gpu(src, vec_rows, vec_cols, new_row, new_col, n, index)
    dst_img = linear_interp_gpu(new_img, kernel)
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
    
    board = 0
    final_img = add_background(dst_img, background_path, board)
    
    rows, cols = src.shape[:2]
    ratio_row = rows / final_img.shape[0]
    ratio_col = cols / final_img.shape[1]
    
    vec_rows[:] = (vec_rows + board) * ratio_row
    vec_cols[:] = (vec_cols + board) * ratio_col
    
    final_img = cv2.resize(final_img, (cols, rows))
    cv2.imwrite(save_path, final_img)
    
    return 1

def generate_a_sample(img_path, background_path, save_path, i=0):
    try:
        max_new_row = 512
        max_new_col = 512
        img = cv2.imread(img_path)
        if img is None:
            return
        new_col = max_new_col
        new_row = max_new_row
        map_rows = np.ones((new_row * new_col,), dtype=np.float32)
        map_cols = np.ones((new_row * new_col,), dtype=np.float32)
        np.random.seed(i)
        kernel = 3
        n = 1
        result = generate_one_img_py(
            img_path, background_path, save_path,
            new_row, new_col, map_rows, map_cols,
            n, i, kernel
        )
        if result == 1:
            map_rows_2d = map_rows.reshape(new_row, new_col)
            map_cols_2d = map_cols.reshape(new_row, new_col)
            mapping_data = np.stack((map_cols_2d, map_rows_2d), axis=2)
            label_dir = os.path.join(os.path.dirname(save_path).replace("images", "labels"))
            os.makedirs(label_dir, exist_ok=True)
            np.save(os.path.join(label_dir, f"{os.path.splitext(os.path.basename(save_path))[0]}.npy"), mapping_data)
    except Exception as e:
        import traceback
        traceback.print_exc()


Found 6184 input images.
Processing [1/5100]: X00016469612.jpg
Processing [2/5100]: X00016469619.jpg
Processing [3/5100]: X00016469620.jpg
Processing [4/5100]: X00016469622.jpg
Processing [5/5100]: X00016469623.jpg
Processing [6/5100]: X00016469669.jpg
Processing [7/5100]: X00016469670.jpg
Processing [8/5100]: X00016469671.jpg
Processing [9/5100]: X00016469672.jpg
Processing [10/5100]: X00016469676.jpg


Traceback (most recent call last):
  File "/tmp/ipykernel_4304/4274363952.py", line 346, in generate_a_sample
    result = generate_one_img_py(
             ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 315, in generate_one_img_py
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 224, in warp_images
    raise ValueError(f"[warp_images] Empty new_img at index {index}")
ValueError: [warp_images] Empty new_img at index 7


Processing [11/5100]: X51005200931.jpg
Processing [12/5100]: X51005200938.jpg
Processing [13/5100]: X51005230605.jpg
Processing [14/5100]: X51005230616.jpg
Processing [15/5100]: X51005230617.jpg
Processing [16/5100]: X51005230648.jpg
Processing [17/5100]: X51005230657.jpg
Processing [18/5100]: X51005230659.jpg
Processing [19/5100]: X51005255805.jpg
Processing [20/5100]: X51005268200.jpg
Processing [21/5100]: X51005268262.jpg
Processing [22/5100]: X51005268275.jpg
Processing [23/5100]: X51005268400.jpg
Processing [24/5100]: X51005268408.jpg
Processing [25/5100]: X51005268472.jpg
Processing [26/5100]: X51005288570.jpg
Processing [27/5100]: X51005301659.jpg
Processing [28/5100]: X51005301661.jpg
Processing [29/5100]: X51005301666.jpg
Processing [30/5100]: X51005301667.jpg
Processing [31/5100]: X51005303661.jpg


Traceback (most recent call last):
  File "/tmp/ipykernel_4304/4274363952.py", line 346, in generate_a_sample
    result = generate_one_img_py(
             ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 315, in generate_one_img_py
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 224, in warp_images
    raise ValueError(f"[warp_images] Empty new_img at index {index}")
ValueError: [warp_images] Empty new_img at index 25


Processing [32/5100]: X51005306399.jpg
Processing [33/5100]: X51005337867.jpg
Processing [34/5100]: X51005337872.jpg
Processing [35/5100]: X51005337877.jpg
Processing [36/5100]: X51005361883.jpg
Processing [37/5100]: X51005361895.jpg
Processing [38/5100]: X51005361912.jpg
Processing [39/5100]: X51005361950.jpg


...

Processing [4983/5100]: X51006334766.jpg
Processing [4984/5100]: X51006334770.jpg
Processing [4985/5100]: X51006334819.jpg
Processing [4986/5100]: X51006334926.jpg
Processing [4987/5100]: X51006334927.jpg
Processing [4988/5100]: X51006335314.jpg
Processing [4989/5100]: X51006335547.jpg
Processing [4990/5100]: X51006335552.jpg
Processing [4991/5100]: X51006335818.jpg


Traceback (most recent call last):
  File "/tmp/ipykernel_4304/4274363952.py", line 346, in generate_a_sample
    result = generate_one_img_py(
             ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 315, in generate_one_img_py
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 224, in warp_images
    raise ValueError(f"[warp_images] Empty new_img at index {index}")
ValueError: [warp_images] Empty new_img at index 4985


Processing [4992/5100]: X51006349081.jpg
Processing [4993/5100]: X51006349083.jpg
Processing [4994/5100]: X51006349085.jpg
Processing [4995/5100]: X51006349086.jpg
Processing [4996/5100]: X51006350737.jpg
Processing [4997/5100]: X51006350750.jpg
Processing [4998/5100]: X51006350763.jpg
Processing [4999/5100]: X51006387660.jpg
Processing [5000/5100]: X51006387737.jpg
Processing [5001/5100]: X51006387740.jpg
Processing [5002/5100]: X51006387812.jpg
Processing [5003/5100]: X51006387813.jpg
Processing [5004/5100]: X51006387847.jpg
Processing [5005/5100]: X51006387850.jpg
Processing [5006/5100]: X51006387931.jpg
Processing [5007/5100]: X51006387932.jpg
Processing [5008/5100]: X51006387950.jpg
Processing [5009/5100]: X51006387951.jpg
Processing [5010/5100]: X51006387953.jpg
Processing [5011/5100]: X51006387971.jpg
Processing [5012/5100]: X51006387973.jpg
Processing [5013/5100]: X51006388044.jpg
Processing [5014/5100]: X51006388064.jpg
Processing [5015/5100]: X51006388065.jpg
Processing [5016

Traceback (most recent call last):
  File "/tmp/ipykernel_4304/4274363952.py", line 346, in generate_a_sample
    result = generate_one_img_py(
             ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 315, in generate_one_img_py
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 224, in warp_images
    raise ValueError(f"[warp_images] Empty new_img at index {index}")
ValueError: [warp_images] Empty new_img at index 5069
Traceback (most recent call last):
  File "/tmp/ipykernel_4304/4274363952.py", line 346, in generate_a_sample
    result = generate_one_img_py(
             ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 315, in generate_one_img_py
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4

Processing [5071/5100]: X51006555835.jpg
Processing [5072/5100]: X51006556591.jpg
Processing [5073/5100]: X51006556596.jpg
Processing [5074/5100]: X51006556604.jpg
Processing [5075/5100]: X51006556610.jpg
Processing [5076/5100]: X51006556646.jpg
Processing [5077/5100]: X51006556648.jpg
Processing [5078/5100]: X51006556650.jpg
Processing [5079/5100]: X51006556654.jpg
Processing [5080/5100]: X51006556656.jpg
Processing [5081/5100]: X51006556657.jpg
Processing [5082/5100]: X51006556658.jpg
Processing [5083/5100]: X51006556723.jpg
Processing [5084/5100]: X51006556726.jpg
Processing [5085/5100]: X51006556728.jpg
Processing [5086/5100]: X51006556730.jpg
Processing [5087/5100]: X51006556732.jpg
Processing [5088/5100]: X51006556734.jpg
Processing [5089/5100]: X51006556738.jpg
Processing [5090/5100]: X51006556740.jpg
Processing [5091/5100]: X51006556806.jpg
Processing [5092/5100]: X51006556808.jpg
Processing [5093/5100]: X51006556809.jpg
Processing [5094/5100]: X51006556810.jpg
Processing [5095

In [ ]:

def batch_process():
    input_dir = "/workspace/scan_images"
    background_dir = "/workspace/backgrounds"
    output_dir = "train_gen"
    start_idx = 0000
    max_images = 5300
    os.makedirs(output_dir, exist_ok=True)
    input_images = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))] * 8

    print(f"Found {len(input_images)} input images.")
    background_images = []
    for root, dirs, files in os.walk(background_dir):
        for file in files:
            if os.path.splitext(file)[1].lower() in {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}:
                background_images.append(os.path.join(root, file))
    if not input_images:
        return
    if not background_images:
        return
    if max_images:
        input_images = input_images[:max_images]
    
    for i, img_name in enumerate(input_images):
        current_idx = start_idx + i
        bg_idx = current_idx % len(background_images)
        img_path = os.path.join(input_dir, img_name)
        background_path = background_images[bg_idx]
        name_without_ext = os.path.splitext(img_name)[0]
        save_dir = os.path.join(output_dir, "images")
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"{current_idx:06d}.jpg")
        
        print(f"Processing [{i+1}/{len(input_images)}]: {img_name}")
        generate_a_sample(img_path, background_path, save_path, current_idx)
    
    print("Batch processing completed!")

if __name__ == "__main__":
    batch_process()

In [19]:

def batch_process():
    input_dir = "/workspace/scan_images"
    background_dir = "/workspace/backgrounds"
    output_dir = "test_gen"
    start_idx = 0000
    max_images = 1400
    os.makedirs(output_dir, exist_ok=True)
    input_images = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))] * 8

    print(f"Found {len(input_images)} input images.")
    background_images = []
    for root, dirs, files in os.walk(background_dir):
        for file in files:
            if os.path.splitext(file)[1].lower() in {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}:
                background_images.append(os.path.join(root, file))
    if not input_images:
        return
    if not background_images:
        return
    if max_images:
        input_images = input_images[:max_images]
    
    for i, img_name in enumerate(input_images):
        current_idx = start_idx + i
        bg_idx = current_idx % len(background_images)
        img_path = os.path.join(input_dir, img_name)
        background_path = background_images[bg_idx]
        name_without_ext = os.path.splitext(img_name)[0]
        save_dir = os.path.join(output_dir, "images")
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"{current_idx:06d}.jpg")
        
        print(f"Processing [{i+1}/{len(input_images)}]: {img_name}")
        generate_a_sample(img_path, background_path, save_path, current_idx)
    
    print("Batch processing completed!")

if __name__ == "__main__":
    batch_process()

Found 6184 input images.
Processing [1/1400]: X00016469612.jpg
Processing [2/1400]: X00016469619.jpg
Processing [3/1400]: X00016469620.jpg
Processing [4/1400]: X00016469622.jpg
Processing [5/1400]: X00016469623.jpg
Processing [6/1400]: X00016469669.jpg
Processing [7/1400]: X00016469670.jpg
Processing [8/1400]: X00016469671.jpg
Processing [9/1400]: X00016469672.jpg
Processing [10/1400]: X00016469676.jpg
Processing [11/1400]: X51005200931.jpg
Processing [12/1400]: X51005200938.jpg
Processing [13/1400]: X51005230605.jpg
Processing [14/1400]: X51005230616.jpg
Processing [15/1400]: X51005230617.jpg
Processing [16/1400]: X51005230648.jpg
Processing [17/1400]: X51005230657.jpg
Processing [18/1400]: X51005230659.jpg
Processing [19/1400]: X51005255805.jpg


Traceback (most recent call last):
  File "/tmp/ipykernel_4304/4274363952.py", line 346, in generate_a_sample
    result = generate_one_img_py(
             ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 315, in generate_one_img_py
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 224, in warp_images
    raise ValueError(f"[warp_images] Empty new_img at index {index}")
ValueError: [warp_images] Empty new_img at index 16
Traceback (most recent call last):
  File "/tmp/ipykernel_4304/4274363952.py", line 346, in generate_a_sample
    result = generate_one_img_py(
             ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 315, in generate_one_img_py
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/427

Processing [20/1400]: X51005268200.jpg
Processing [21/1400]: X51005268262.jpg
Processing [22/1400]: X51005268275.jpg
Processing [23/1400]: X51005268400.jpg
Processing [24/1400]: X51005268408.jpg
Processing [25/1400]: X51005268472.jpg
Processing [26/1400]: X51005288570.jpg
Processing [27/1400]: X51005301659.jpg
Processing [28/1400]: X51005301661.jpg
Processing [29/1400]: X51005301666.jpg
Processing [30/1400]: X51005301667.jpg
Processing [31/1400]: X51005303661.jpg
Processing [32/1400]: X51005306399.jpg
Processing [33/1400]: X51005337867.jpg
Processing [34/1400]: X51005337872.jpg
Processing [35/1400]: X51005337877.jpg
Processing [36/1400]: X51005361883.jpg
Processing [37/1400]: X51005361895.jpg
Processing [38/1400]: X51005361912.jpg
Processing [39/1400]: X51005361950.jpg
Processing [40/1400]: X51005365179.jpg
Processing [41/1400]: X51005365187.jpg
Processing [42/1400]: X51005433518.jpg


Traceback (most recent call last):
  File "/tmp/ipykernel_4304/4274363952.py", line 346, in generate_a_sample
    result = generate_one_img_py(
             ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 315, in generate_one_img_py
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 224, in warp_images
    raise ValueError(f"[warp_images] Empty new_img at index {index}")
ValueError: [warp_images] Empty new_img at index 35
Traceback (most recent call last):
  File "/tmp/ipykernel_4304/4274363952.py", line 346, in generate_a_sample
    result = generate_one_img_py(
             ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 315, in generate_one_img_py
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/427

Processing [43/1400]: X51005441398.jpg
Processing [44/1400]: X51005441401.jpg
Processing [45/1400]: X51005441402.jpg
Processing [46/1400]: X51005441407.jpg
Processing [47/1400]: X51005441408.jpg
Processing [48/1400]: X51005444033.jpg
Processing [49/1400]: X51005444037.jpg
Processing [50/1400]: X51005444040.jpg
Processing [51/1400]: X51005444041.jpg


...

Traceback (most recent call last):
  File "/tmp/ipykernel_4304/4274363952.py", line 346, in generate_a_sample
    result = generate_one_img_py(
             ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 315, in generate_one_img_py
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 224, in warp_images
    raise ValueError(f"[warp_images] Empty new_img at index {index}")
ValueError: [warp_images] Empty new_img at index 1330


Processing [1337/1400]: X51006620437.jpg
Processing [1338/1400]: X51006647932.jpg
Processing [1339/1400]: X51006647933.jpg
Processing [1340/1400]: X51006647984.jpg
Processing [1341/1400]: X51006679216.jpg
Processing [1342/1400]: X51006679222.jpg
Processing [1343/1400]: X51006713996.jpg
Processing [1344/1400]: X51006714065.jpg
Processing [1345/1400]: X51006733495.jpg
Processing [1346/1400]: X51006828199.jpg
Processing [1347/1400]: X51006828200.jpg
Processing [1348/1400]: X51006828201.jpg
Processing [1349/1400]: X51006828217.jpg
Processing [1350/1400]: X51006856981.jpg
Processing [1351/1400]: X51006856982.jpg
Processing [1352/1400]: X51006857071.jpg
Processing [1353/1400]: X51006857126.jpg
Processing [1354/1400]: X51006857132.jpg
Processing [1355/1400]: X51006857137.jpg
Processing [1356/1400]: X51006857265.jpg
Processing [1357/1400]: X51006857540.jpg
Processing [1358/1400]: X51006857660.jpg
Processing [1359/1400]: X51006867435.jpg
Processing [1360/1400]: X51006867436.jpg
Processing [1361

Traceback (most recent call last):
  File "/tmp/ipykernel_4304/4274363952.py", line 346, in generate_a_sample
    result = generate_one_img_py(
             ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 315, in generate_one_img_py
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 224, in warp_images
    raise ValueError(f"[warp_images] Empty new_img at index {index}")
ValueError: [warp_images] Empty new_img at index 1377


Processing [1388/1400]: X51007262330.jpg
Processing [1389/1400]: X51007339094.jpg
Processing [1390/1400]: X51007339095.jpg
Processing [1391/1400]: X51007339097.jpg
Processing [1392/1400]: X51007339098.jpg
Processing [1393/1400]: X51007339100.jpg
Processing [1394/1400]: X51007339105.jpg
Processing [1395/1400]: X51007339106.jpg
Processing [1396/1400]: X51007339107.jpg
Processing [1397/1400]: X51007339109.jpg
Processing [1398/1400]: X51007339110.jpg
Processing [1399/1400]: X51007339111.jpg
Processing [1400/1400]: X51007339112.jpg
Batch processing completed!


Traceback (most recent call last):
  File "/tmp/ipykernel_4304/4274363952.py", line 346, in generate_a_sample
    result = generate_one_img_py(
             ^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 315, in generate_one_img_py
    dst_img = warp_images(dst_img, src, vec_rows, vec_cols, index)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_4304/4274363952.py", line 224, in warp_images
    raise ValueError(f"[warp_images] Empty new_img at index {index}")
ValueError: [warp_images] Empty new_img at index 1399


In [15]:
import os

def check_dir(path):
    print(f"\n📂 Kiểm tra thư mục: {path}")
    if not os.path.exists(path):
        print("❌ Thư mục không tồn tại.")
        return
    
    subdirs = []
    files = []
    
    for root, dirs, filenames in os.walk(path):
        for d in dirs:
            subdirs.append(os.path.join(root, d))
        for f in filenames:
            files.append(os.path.join(root, f))
    
    print(f"  📁 Số thư mục con: {len(subdirs)}")
    for d in subdirs[:10]:  # chỉ in 10 thư mục đầu
        print("   -", d)
    if len(subdirs) > 10:
        print("   ...")

    print(f"  📄 Số file: {len(files)}")
    for f in files[:10]:  # chỉ in 10 file đầu
        print("   -", f)
    if len(files) > 10:
        print("   ...")

# Kiểm tra 2 thư mục
check_dir("/workspace/train_gen/images")
check_dir("/workspace/train_gen/labels")
check_dir("/workspace/test_gen/images")
check_dir("/workspace/tesst_gen/labels")



📂 Kiểm tra thư mục: /workspace/train_gen/images
  📁 Số thư mục con: 0
  📄 Số file: 4774
   - /workspace/train_gen/images/000000.jpg
   - /workspace/train_gen/images/000001.jpg
   - /workspace/train_gen/images/000002.jpg
   - /workspace/train_gen/images/000003.jpg
   - /workspace/train_gen/images/000004.jpg
   - /workspace/train_gen/images/000005.jpg
   - /workspace/train_gen/images/000007.jpg
   - /workspace/train_gen/images/000008.jpg
   - /workspace/train_gen/images/000009.jpg
   - /workspace/train_gen/images/000011.jpg
   ...

📂 Kiểm tra thư mục: /workspace/train_gen/labels
  📁 Số thư mục con: 0
  📄 Số file: 4774
   - /workspace/train_gen/labels/000000.npy
   - /workspace/train_gen/labels/000001.npy
   - /workspace/train_gen/labels/000002.npy
   - /workspace/train_gen/labels/000003.npy
   - /workspace/train_gen/labels/000004.npy
   - /workspace/train_gen/labels/000005.npy
   - /workspace/train_gen/labels/000007.npy
   - /workspace/train_gen/labels/000008.npy
   - /workspace/train_g

In [17]:
import os
import random
import shutil

def reduce_dataset(images_dir, labels_dir, keep_n=1000):
    # Lấy danh sách file ảnh và nhãn
    image_files = sorted([f for f in os.listdir(images_dir) if f.endswith(".jpg")])
    label_files = sorted([f for f in os.listdir(labels_dir) if f.endswith(".npy")])

    assert len(image_files) == len(label_files), "Số lượng ảnh và nhãn không khớp!"

    total = len(image_files)
    print(f"Tổng số dữ liệu: {total}")

    if keep_n >= total:
        print("⚠️ keep_n >= total, không cần xóa gì.")
        return

    # Chọn ngẫu nhiên các chỉ số để giữ lại
    keep_indices = set(random.sample(range(total), keep_n))

    for idx, (img, lbl) in enumerate(zip(image_files, label_files)):
        if idx not in keep_indices:
            os.remove(os.path.join(images_dir, img))
            os.remove(os.path.join(labels_dir, lbl))

    print(f"✅ Giữ lại {keep_n}/{total} mẫu, đã xóa {total - keep_n} mẫu.")

# Ví dụ dùng cho test_gen
reduce_dataset("/workspace/test_gen/images", "/workspace/test_gen/labels", keep_n=0)

# Ví dụ dùng cho train_gen
reduce_dataset("/workspace/train_gen/images", "/workspace/train_gen/labels", keep_n=0)



Tổng số dữ liệu: 1245
✅ Giữ lại 0/1245 mẫu, đã xóa 1245 mẫu.
Tổng số dữ liệu: 1200
✅ Giữ lại 0/1200 mẫu, đã xóa 1200 mẫu.


In [9]:
import os

image_dir = "test_gen/images"
label_dir = "test_gen/labels"

# --- Dọn ảnh không có label ---
image_files = sorted(os.listdir(image_dir))
label_files = set(os.listdir(label_dir))

for img_file in image_files:
    img_path = os.path.join(image_dir, img_file)
    if not os.path.isfile(img_path):
        continue
    name, ext = os.path.splitext(img_file)
    expected_label = f"{name}.npy"
    if expected_label not in label_files:
        print(f"Xóa ảnh {img_file} vì không có label tương ứng")
        os.remove(img_path)

# --- Dọn label không có ảnh ---
label_files = sorted(os.listdir(label_dir))
image_files_set = set(os.listdir(image_dir))

for lbl_file in label_files:
    lbl_path = os.path.join(label_dir, lbl_file)
    if not os.path.isfile(lbl_path):
        continue
    name, ext = os.path.splitext(lbl_file)
    expected_image = f"{name}.jpg"  # hoặc .png tùy ảnh
    if expected_image not in image_files_set:
        print(f"Xóa label {lbl_file} vì không có ảnh tương ứng")
        os.remove(lbl_path)

print("Dataset đã dọn xong")


Dataset đã dọn xong


# Clean

In [25]:
import os

def check_dir(path):
    print(f"\n📂 Kiểm tra thư mục: {path}")
    if not os.path.exists(path):
        print("❌ Thư mục không tồn tại.")
        return
    
    subdirs = []
    files = []
    
    for root, dirs, filenames in os.walk(path):
        for d in dirs:
            subdirs.append(os.path.join(root, d))
        for f in filenames:
            files.append(os.path.join(root, f))
    
    print(f"  📁 Số thư mục con: {len(subdirs)}")
    for d in subdirs[:10]:  # chỉ in 10 thư mục đầu
        print("   -", d)
    if len(subdirs) > 10:
        print("   ...")

    print(f"  📄 Số file: {len(files)}")
    for f in files[:10]:  # chỉ in 10 file đầu
        print("   -", f)
    if len(files) > 10:
        print("   ...")

# Kiểm tra 2 thư mục
check_dir("/workspace/train_gen/images")
check_dir("/workspace/train_gen/labels")
check_dir("/workspace/test_gen/images")
check_dir("/workspace/test_gen/labels")



📂 Kiểm tra thư mục: /workspace/train_gen/images
  📁 Số thư mục con: 0
  📄 Số file: 4800
   - /workspace/train_gen/images/000000.jpg
   - /workspace/train_gen/images/000001.jpg
   - /workspace/train_gen/images/000002.jpg
   - /workspace/train_gen/images/000003.jpg
   - /workspace/train_gen/images/000004.jpg
   - /workspace/train_gen/images/000005.jpg
   - /workspace/train_gen/images/000006.jpg
   - /workspace/train_gen/images/000008.jpg
   - /workspace/train_gen/images/000009.jpg
   - /workspace/train_gen/images/000010.jpg
   ...

📂 Kiểm tra thư mục: /workspace/train_gen/labels
  📁 Số thư mục con: 0
  📄 Số file: 4800
   - /workspace/train_gen/labels/000000.npy
   - /workspace/train_gen/labels/000001.npy
   - /workspace/train_gen/labels/000002.npy
   - /workspace/train_gen/labels/000003.npy
   - /workspace/train_gen/labels/000004.npy
   - /workspace/train_gen/labels/000005.npy
   - /workspace/train_gen/labels/000006.npy
   - /workspace/train_gen/labels/000008.npy
   - /workspace/train_g

In [23]:
import os

image_dir = "test_gen/images"
label_dir = "test_gen/labels"

# --- Dọn ảnh không có label ---
image_files = sorted(os.listdir(image_dir))
label_files = set(os.listdir(label_dir))

for img_file in image_files:
    img_path = os.path.join(image_dir, img_file)
    if not os.path.isfile(img_path):
        continue
    name, ext = os.path.splitext(img_file)
    expected_label = f"{name}.npy"
    if expected_label not in label_files:
        print(f"Xóa ảnh {img_file} vì không có label tương ứng")
        os.remove(img_path)

# --- Dọn label không có ảnh ---
label_files = sorted(os.listdir(label_dir))
image_files_set = set(os.listdir(image_dir))

for lbl_file in label_files:
    lbl_path = os.path.join(label_dir, lbl_file)
    if not os.path.isfile(lbl_path):
        continue
    name, ext = os.path.splitext(lbl_file)
    expected_image = f"{name}.jpg"  # hoặc .png tùy ảnh
    if expected_image not in image_files_set:
        print(f"Xóa label {lbl_file} vì không có ảnh tương ứng")
        os.remove(lbl_path)

print("Dataset đã dọn xong")


Dataset đã dọn xong


In [24]:
import os
import random
import shutil

def reduce_dataset(images_dir, labels_dir, keep_n=1000):
    # Lấy danh sách file ảnh và nhãn
    image_files = sorted([f for f in os.listdir(images_dir) if f.endswith(".jpg")])
    label_files = sorted([f for f in os.listdir(labels_dir) if f.endswith(".npy")])

    assert len(image_files) == len(label_files), "Số lượng ảnh và nhãn không khớp!"

    total = len(image_files)
    print(f"Tổng số dữ liệu: {total}")

    if keep_n >= total:
        print("⚠️ keep_n >= total, không cần xóa gì.")
        return

    # Chọn ngẫu nhiên các chỉ số để giữ lại
    keep_indices = set(random.sample(range(total), keep_n))

    for idx, (img, lbl) in enumerate(zip(image_files, label_files)):
        if idx not in keep_indices:
            os.remove(os.path.join(images_dir, img))
            os.remove(os.path.join(labels_dir, lbl))

    print(f"✅ Giữ lại {keep_n}/{total} mẫu, đã xóa {total - keep_n} mẫu.")

# Ví dụ dùng cho test_gen
reduce_dataset("/workspace/test_gen/images", "/workspace/test_gen/labels", keep_n=1200)

# Ví dụ dùng cho train_gen
reduce_dataset("/workspace/train_gen/images", "/workspace/train_gen/labels", keep_n=4800)



Tổng số dữ liệu: 1342
✅ Giữ lại 1200/1342 mẫu, đã xóa 142 mẫu.
Tổng số dữ liệu: 4883
✅ Giữ lại 4800/4883 mẫu, đã xóa 83 mẫu.
